# Interactive 2025 comparison for signal 2B100
Aggregate both sources by floored date (handles values at 23:00), keep only 2025 values, and plot them together with hover tooltips.

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Fallback renderer if nbformat is missing (avoids ValueError)
#try:
import nbformat  # noqa: F401
#except ImportError:
#    pio.renderers.default = "browser"

# Read and normalize dates to midnight for safe aggregation
pbi = pd.read_csv("PBI_2B100.csv", parse_dates=["Date"])
psu = pd.read_csv("PSU_2B100.csv", parse_dates=["DateTime"])

#pbi["date"] = pbi["Date"].dt.floor("D")
#psu["date"] = psu["DateTime"].dt.floor("D")
pbi["date"] = pbi["Date"].dt.ceil("D")
psu["date"] = psu["DateTime"].dt.ceil("D")

# Keep only 2025 rows for PBI, aggregate by date, and keep day-of-year for alignment
pbi_2025 = (
    pbi[pbi["date"].dt.year == 2025]
    .groupby("date", as_index=False)["Ped_Volumes"]
    .sum()
    .rename(columns={"Ped_Volumes": "PBI_Ped_Volumes"})
)
pbi_2025["day_of_year"] = pbi_2025["date"].dt.dayofyear

# PSU stores 2025 values in a column; aggregate by floored date and align by day-of-year
psu_2025 = (
    psu.groupby("date", as_index=False)["2025"]
    .sum()
    .rename(columns={"2025": "PSU_Ped_Volumes"})
)
psu_2025["day_of_year"] = psu_2025["date"].dt.dayofyear

# Align on day-of-year so series share the same x-axis even if the stored years differ
merged = (
    pd.merge(
        pbi_2025[["date", "day_of_year", "PBI_Ped_Volumes"]],
        psu_2025[["day_of_year", "PSU_Ped_Volumes"]],
        on="day_of_year",
        how="inner",
    )
    .sort_values("date")
    .reset_index(drop=True)
)

# Build a canonical 2025 date from day-of-year for clean tooltips
merged["date_2025"] = pd.to_datetime("2025-01-01") + pd.to_timedelta(merged["day_of_year"] - 1, unit="D")

merged.head()

,date,day_of_year,PBI_Ped_Volumes,PSU_Ped_Volumes,date_2025
0,2025-01-01,1,450.1,444.38,2025-01-01
1,2025-01-01,1,450.1,0.00,2025-01-01
2,2025-01-02,2,607.1,592.76,2025-01-02
3,2025-01-03,3,748.6,734.71,2025-01-03
4,2025-01-04,4,591.4,580.90,2025-01-04


In [2]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=merged["date_2025"],
        y=merged["PBI_Ped_Volumes"],
        mode="lines+markers",
        name="PBI 2025",
        hovertemplate="Date: %{x|%Y-%m-%d}<br>PBI: %{y:.2f}<extra></extra>",
    )
)

fig.add_trace(
    go.Scatter(
        x=merged["date_2025"],
        y=merged["PSU_Ped_Volumes"],
        mode="lines+markers",
        name="PSU 2025",
        hovertemplate="Date: %{x|%Y-%m-%d}<br>PSU: %{y:.2f}<extra></extra>",
    )
)

fig.update_layout(
    title="PBI vs PSU pedestrian volumes for 2025 (Signal 2B100)",
    xaxis_title="Date",
    yaxis_title="Pedestrian volumes",
    hovermode="x unified",
    legend_title="Source",
)

fig.show()

# Load Historical Data

In [36]:
import duckdb

con = duckdb.connect()

ped_joined = con.sql("""
    SELECT *
    FROM 'Z:\\Data\\Performance\\Ped\\*.parquet'
    NATURAL INNER JOIN 'Z:\\Data\\Performance\\Unique_Ped_Actuations\\*.parquet'
    WHERE deviceid = 902
""").df()

ped_joined


,PedServices,PedActuation,TimeStamp,DeviceID,Phase,Unique_Actuations
0,2,5,2022-01-02 00:15:00,902,2,1
1,2,2,2022-01-02 00:15:00,902,4,1
2,2,4,2022-01-02 00:15:00,902,6,1
3,3,5,2022-01-02 00:15:00,902,8,4
4,1,2,2022-01-02 01:30:00,902,4,1
...,...,...,...,...,...,...
210308,2,2,2024-05-11 23:30:00,902,6,2
210309,1,2,2024-05-11 23:30:00,902,8,1
210310,1,1,2024-05-11 23:45:00,902,2,1
210311,1,3,2024-05-11 23:45:00,902,4,1


In [50]:
min_time = ped_joined["TimeStamp"].min()
max_time = ped_joined["TimeStamp"].max()

# Create range from min to max in 60-minute intervals
time_range = pd.date_range(start=min_time, end=max_time, freq='60min')

# Get distinct Phase values
phase_values = ped_joined["Phase"].unique()

# Create DataFrame with all combinations of TimeStamp and Phase
time_range_df = pd.DataFrame(
    pd.MultiIndex.from_product([time_range, phase_values], names=['TimeStamp', 'Phase']).tolist(),
    columns=['TimeStamp', 'Phase']
)

# Left join and aggregate
hourly_ped = con.sql("""
    SELECT time_range_df.TimeStamp AS TimeStamp,
        time_range_df.Phase AS Phase,
        COALESCE(SUM(Unique_Actuations), 0) AS Unique_Actuations
    FROM time_range_df
    LEFT JOIN ped_joined ON time_range_df.TimeStamp = ped_joined.TimeStamp 
        AND time_range_df.Phase = ped_joined.Phase
    GROUP BY time_range_df.TimeStamp, time_range_df.Phase
""").df()

hourly_ped

,TimeStamp,Phase,Unique_Actuations
0,2023-06-18 08:45:00,4,1.0
1,2023-06-18 10:45:00,6,1.0
2,2023-06-18 11:45:00,2,2.0
3,2023-06-21 06:45:00,4,1.0
4,2023-06-21 10:45:00,4,2.0
...,...,...,...
85875,2024-06-11 04:45:00,8,0.0
85876,2024-06-12 02:45:00,8,0.0
85877,2024-06-13 01:45:00,6,0.0
85878,2024-06-13 02:45:00,2,0.0


In [52]:
# Add Ped Volumes
#(0.7167 * Unique_Actuations_Rolling + 0.0599 * Unique_Actuations_Rolling^2)::FLOAT AS Estimated_Hourly,1.1063

ped_joined = con.sql("""
    SELECT TimeStamp, Phase,
    (0.7167 * Unique_Actuations + 0.0599 * Unique_Actuations^2 + 1.1063) AS Volumes
    FROM hourly_ped
    ORDER BY TimeStamp, Phase
""").df()

ped_joined

,TimeStamp,Phase,Volumes
0,2022-01-01 00:45:00,2,1.8829
1,2022-01-01 00:45:00,4,1.1063
2,2022-01-01 00:45:00,6,1.1063
3,2022-01-01 00:45:00,8,1.8829
4,2022-01-01 01:45:00,2,1.1063
...,...,...,...
85875,2024-06-13 12:45:00,8,7.5629
85876,2024-06-13 13:45:00,2,6.1873
85877,2024-06-13 13:45:00,4,3.7955
85878,2024-06-13 13:45:00,6,7.5629
